In [ ]:
#!pip install mysql-connector-python

import os
import pandas as pd
import mysql.connector
from  mysql.connector import errorcode
import ast


In [ ]:
def limpieza_datos_actores_pelis():
    """
    Esta función transforma un csv en una lista de tuplas de actores por pelicula.
    
    Args:
    - No recibe parametros.

    Returns:
    - devuelve una lista de tuplas (id pelicula, nombre actor).
    """
    lista_tuplas_actores =[]
    lista_final_actores = []

    for i in range(1990,2024):
        try:
            df_pelis = pd.read_csv(f'datos/datos_peliculas/lista_peliculas_{i}_1000.csv')
        except:
            print(f"lista_peliculas_{i}_1000.csv - no existe")            

        orden_actores = ["0","7"]

        df_actores_pelis = df_pelis.reindex(columns = orden_actores)

        # tupla_actores
        for fila in df_actores_pelis.values:
            try:            
                linea = ast.literal_eval(fila[1])
            except:
                linea = 'No data'
            for elemento in linea[1:]:
                lista_tuplas_actores.append((fila[0], elemento))

    df_actores = pd.DataFrame(set(lista_tuplas_actores), columns =['id', 'nombre'])

    #quitar nan y null del dataframe
    df_final_actores = df_actores.dropna(how='any', axis=0) 

    for fila in df_final_actores.values:
        lista_final_actores.append(tuple(fila))     
            
    return lista_final_actores           

In [ ]:
def insertar_actores_pelis(lista):
    """
    Esta función toma una lista de tuplas y la inserta en una tabla de la bbdd
    
    Args:
    - lista: listade tuplas donde cada tupla incluye los datos de un actor: id pelicula, nombre y apellido del actor

    Returns:
    - esta funcion devuelve el número de datos insertados

    """
    contrasena = os.environ.get('contraseña')

    # Ejecutar solo cuando la tabla de detalles_interpretes esté con los datos
    #conexion     
    cnx = mysql.connector.connect(user="root",
                            password=contrasena,
                            host="localhost", database = "bho_peliculas_3")

    mycursor = cnx.cursor()

    #tabla interpretes por pelicula
    sql_interpretes_peliculas = "INSERT INTO interpretes_peliculas (id_pelicula, nombre_interprete) VALUES (%s, %s)"

    try:
        mycursor.executemany(sql_interpretes_peliculas, lista)
        cnx.commit()
        print(mycursor.rowcount, "registros insertados")
    except mysql.connector.Error as err:
                print(err)
                print("Error Code:", err.errno)
                print("SQLSTATE", err.sqlstate)
                print("Message", err.msg)
                print("peliculas - error inserción")  

    cnx.close()    

    return mycursor.rowcount

In [ ]:
#llamar a las funciones para limpiar los datos e insertarlos en la bbdd
insertar_actores_pelis(limpieza_datos_actores_pelis())    